In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/My Drive/CS492I/final project/kobert"
!ls

from pathlib import Path

root = Path("/content/drive/My Drive/CS492I/final project/kobert")
print(root)

/content/drive/My Drive/CS492I/final project/kobert
data.csv
/content/drive/My Drive/CS492I/final project/kobert


In [ ]:
!pip install gluonnlp==0.10.0 pandas tqdm   
!pip install mxnet==1.8.0
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

     |████████████████████████████████| 344 kB 8.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595731 sha256=05c5f58a59cfaaa20dabcff8cfbfafbf7a6e239b4f462d7fe21af0dcfb6a506f
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 38.7 MB 180 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 1.1 MB 8.2 MB/s 
     |████████████████████████████████| 2.5 MB 9.8 MB/s 
     |████████████████████████████████| 895 kB 57.4 MB/s 
     |████████████████████████████████| 3.3 MB 41.0 MB/s 


In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-78t7_hvg/kobert-tokenizer_c35fe9b661f04ea49c8d1cffd4b33f39
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-78t7_hvg/kobert-tokenizer_c35fe9b661f04ea49c8d1cffd4b33f39
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4682 sha256=ce1df50ff7fb5b88be374287664b7dcb40548e945c48f0f666fff054eaa0c4f8
  Stored in directory: /tmp/pip-ephem-wheel-cache-0oddqlrn/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-k741574c
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-k741574c
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=43b71d78276d8b31db5c89b82f5c1ccff55adbf8f826d38257f64e4fc77eec6b
  Stored in directory: /tmp/pip-ephem-wheel-cache-38fa_s96/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [ ]:
from kobert.pytorch_kobert import get_kobert_model
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, notebook
import pandas as pd

In [ ]:
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

device = torch.device("cuda:0") if torch.cuda.is_available() else "cpu"

In [ ]:
import csv

data_list = []

with open("data.csv") as f:
  reader = csv.reader(f)
  for row in reader:
    data_list.append(row)

data_list

[['뭐를 제일 많이 했냐면은 메이플 스토리 스타크래프트 그리고 귀혼 아니 귀혼 내가 롤플레잉 이런 거 좋아했지.', '1'],
 ['써든어택도 많이 했고 스페셜 포스 이런 것도 했고 워록이라고 또 있어 넥슨 거인데 워록 알지', '1'],
 ['총 쏘고 뭐~ 수류탄 던지고 칼 썰고 이게 끝인데 워록은 탈 수가 있었어 탱크 타고 비행기 타고', '1'],
 ['스무 살 넘어서도 계속했던 게임이 메이플 스토리 여자 친구랑 오버워치 조금 자주 하고', '1'],
 ['메이플 스토리 계속 이제 키우다 보면은 애정이 가잖아 내 캐릭터에 난 그런 걸 조금 좋아해.', '1'],
 ['가 그런 게임들 있지 않냐 나는 그런 거 잘 안 해봤거든.', '1'],
 ['메이플 많이 했지 너는 어떤 게임 많이 해봤니?', '1'],
 ['나는 진짜 다양하게 게임 많이 했지 아까 네가 얘기하 했던 거 다 해봤고', '1'],
 ['-이- 이 차 전직 그러니까 그때 지금은 모르겠는데 지금은 레벨이 너무 빨라 가지고 지금은 그런 구간이 없어졌는데 옛날에', '1'],
 ['요즘은 날아다니는데 막 지금 그냥 진짜 걸어가고 막 하는 그런 감성 (()) 딱 진짜 시그니스인가 그때까지만 해도', '1'],
 ['그런 게 별로 없었잖아 날아다니고 하는 게 별로 없었잖아 그거 때쯤에 이제 내가', '1'],
 ['몬스터가 많이 바뀌었잖아 레벨이랑 그런 거 하고 뭐~ 메이플 월드도 많이 바뀌었고', '1'],
 ['스타크래프트 유즈맵 뭐~ 그런 것도 많이 하고 막 그랬는데 애들이 나 빼고 다 같이 시작한 게임이 롤이었어 고등학교 때', '1'],
 ['되게 재밌어 보이긴 한데 조금 어려워 보여 그래 갖고 진입 장벽이 조금 높잖아 롤이 처음에는 그래 가지고 처음에 접할기가 조금 무서웠지 근데 이제',
  '1'],
 ['밤새 이제 피시를 많이 갔는데 안피를 갔는데 아~ 애들이 너무 재밌게 하고 있는 거야 내 혼자 뭔가 소외감이 그~ 어뜨케 하는 거야 물어봤는데 막',
  '1'],
 ['아~ 너 롤

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 128
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5  
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
tok=tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

def save_model(model, mode="last"):
    torch.save(model.state_dict(), root / f'{type(model).__name__}_{mode}.ckpt')

In [ ]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

    save_model(model, mode=str(e))

save_model(model)

  0%|          | 0/2966 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.3568780422210693 train acc 0.3125
epoch 1 batch id 201 loss 1.2256475687026978 train acc 0.35789800995024873
epoch 1 batch id 401 loss 1.0212366580963135 train acc 0.4262390897755611
epoch 1 batch id 601 loss 0.9519760608673096 train acc 0.47129783693843597
epoch 1 batch id 801 loss 0.9010180830955505 train acc 0.5001560549313359
epoch 1 batch id 1001 loss 1.067941665649414 train acc 0.5215721778221778
epoch 1 batch id 1201 loss 0.8256863355636597 train acc 0.5375728559533722
epoch 1 batch id 1401 loss 0.8106166124343872 train acc 0.5490609386152748
epoch 1 batch id 1601 loss 0.9049674272537231 train acc 0.5576983135540288
epoch 1 batch id 1801 loss 0.791877806186676 train acc 0.5657967795669073
epoch 1 batch id 2001 loss 1.0802305936813354 train acc 0.5732133933033483
epoch 1 batch id 2201 loss 0.6558390259742737 train acc 0.579835870059064
epoch 1 batch id 2401 loss 1.0329819917678833 train acc 0.5857455226988755
epoch 1 batch id 2601 loss 0.6785759925842285

  0%|          | 0/742 [00:00<?, ?it/s]

epoch 1 test acc 0.6695995025368637


  0%|          | 0/2966 [00:00<?, ?it/s]

...

epoch 4 test acc 0.6819506104328523


  0%|          | 0/2966 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.48357081413269043 train acc 0.796875
epoch 5 batch id 201 loss 0.38165777921676636 train acc 0.8165422885572139
epoch 5 batch id 401 loss 0.48225533962249756 train acc 0.8176823566084788
epoch 5 batch id 601 loss 0.39304688572883606 train acc 0.8203514975041597
epoch 5 batch id 801 loss 0.528519332408905 train acc 0.8230532147315855
epoch 5 batch id 1001 loss 0.45522502064704895 train acc 0.8248001998001998
epoch 5 batch id 1201 loss 0.29900625348091125 train acc 0.82733139050791
epoch 5 batch id 1401 loss 0.38448867201805115 train acc 0.8282030692362599
epoch 5 batch id 1601 loss 0.23035287857055664 train acc 0.8295986883198001
epoch 5 batch id 1801 loss 0.21818052232265472 train acc 0.8294176846196557
epoch 5 batch id 2001 loss 0.5404507517814636 train acc 0.8303660669665167
epoch 5 batch id 2201 loss 0.36460745334625244 train acc 0.8313976601544752
epoch 5 batch id 2401 loss 0.5115378499031067 train acc 0.8321662848812995
epoch 5 batch id 2601 loss 0.295200

  0%|          | 0/742 [00:00<?, ?it/s]

epoch 5 test acc 0.6862625356746472


In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 안녕!


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 혐오가 느껴집니다.


하고싶은 말을 입력해주세요 : 바보
>> 입력하신 내용에서 혐오가 느껴집니다.




KeyboardInterrupt: ignored